### In this notebook I want make one of the A2 colour plots for a random ising model problems where we separate the qubits into two classes based on their $h_i$ value. We will have a hi class and a lo class of qubtis

From a randomly selected optimisation problem where we generate $h_i$ and $J_{ij}$. Where the group of N qubits in the problem are $Q$ and the high class is $Q_{hi}$ and the low class is $Q_{lo}$. Qubit $i$ will be in the high class if its h value is greater than the median value. If there is an odd number of qubits, then the median qubit is assigned a class based on whether it's value is closer to the average value of the other elements in the $Q_{hi}$ group or or the $Q_{lo}$ group. 

In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Qubits_test = 4
scale_test = 3.
h_test = np.random.normal(0,scale_test,Qubits_test)
J_test =  np.random.normal(0,scale_test,(Qubits_test,Qubits_test))
sorted_h_test = np.sort(h_test)
if Qubits_test%2 == 0: # if even
    h_div_test = np.average([sorted_h_test[Qubits_test/2 - 1],sorted_h_test[Qubits_test/2]])
elif Qubits_test%2 == 1: # if odd
    h_div_test = np.average([np.average(sorted_h_test[:(Qubits_test-1)/2]),np.average(sorted_h_test[(Qubits_test+1)/2:])])
Q_hi_test = np.where(h_test>=h_div_test)[0]
Q_lo_test = np.where(h_test<h_div_test)[0]
print Q_hi_test
print Q_lo_test

In [ ]:
# have a signle run before we move on to do the full scope examination
Qubits = 8
scale = 3.
h_i = np.random.normal(0,scale,Qubits)
J_ij =  np.random.normal(0,scale,(Qubits,Qubits))
sorted_h_i = np.sort(h_i)
if Qubits%2 == 0: # if even
    h_i_div = np.average([sorted_h_i[Qubits/2 - 1],sorted_h_i[Qubits/2]])
elif Qubits%2 == 1: # if odd
    h_i_div = np.average([np.average(sorted_h_i[:(Qubits-1)/2]),np.average(sorted_h_i[(Qubits+1)/2:])])
Q_hi = np.where(h_i>=h_i_div)[0]
Q_lo = np.where(h_i<h_i_div)[0]
print Q_hi
print Q_lo


#now lets run it
a = mylib.Anneal(Qubits,[h_i,J_ij],T=10,points = 1000)
a.run()
a.show_results()

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\Jup010 - Anneal_path_graph.png")
plt.figure(figsize=(6,6)); plt.imshow(img1)
plt.axis('off')
plt.show()

I want to carry out a bunch of anneals where the annealing schedule A path with have the above form except substituting that the log --> hi and anc --> lo. I want them to be simply a linear schedule where they are related by
\begin{equation}
    A_{lo}(s)=(1-s) \sin(\theta)
\end{equation}

\begin{equation}
    A_{hi}(s)=(1-s) \cos(\theta)
\end{equation}

\begin{equation}
    \text{and so }\\
    \sqrt{A_{lo}^2(s)+A_{hi}^2(s)}=(1-s)
\end{equation}

In [ ]:
# This insures that we complete a small number of points that cover the theta range first. Then we go for more detail.
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))
print thetas_unordered

In [ ]:
# We are going to use the h_i and J_ij parameters used in the example above. This is so we can have prechecked a 
# single run with these parameters to make sure they don't lead to a solution far from what I am expecting
J_ij = np.triu(J_ij, k =1) # for tidyness
print h_i
print J_ij

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200
an = {}
last_prob = []
for x in thetas_unordered:
    sfhi = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sflo = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfhi if n in Q_hi else sflo for n in range(Qubits)],[sfB]]
    a = mylib.Anneal(Qubits,[h_i,J_ij],T=10,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter022.001')

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
#      HERE I WANT TO DEFINE A FUNCTION THAT RETURNS THE HAMILTONIAN AT ANY POINT ON A2 SPACE
# ------------------------------------------------------------------------------------------------------------------------------
'''We need this to be specific to the linear schedule. So for each point we will need to calculate the angle of theta and
thus calculate s so we can feed this into B(s). 
I want to be able create a plot that has sqrt(A_log**2+A_anc**2) = 1 since these are the point that my previous work has 
passed through'''
# need to load back in the an that I created before. Pick the one from the centre [with theta = 45 degrees].
a =  mylib.load_object('Jupyter022.001')['theta=%s'%(np.pi/4.)]
nax = 1000
sp = np.linspace(0, 1, nax)
gaps = np.zeros((nax,nax))
def H_A2((x,y),s):
    return sum([a.Hb[i]*A for i,A in enumerate([x if n in Q_hi else y for n in range(Qubits)])])+s*a.Hp

def gap_finder(M):
    d,P,P_in = mylib.Diagonaliser(M)
    return min(d[d!=min(d)])-min(d)


def gap_A2((x,y)):
    s = 1-(x**2+y**2)
    if s<0: return 0
    else: return gap_finder(H_A2((x,y),s))

for num_x,x in enumerate(sp):
    print 'running %s of %s'%(num_x,nax) #should have been (num_x,nax-1) but ahh-well
    for num_y,y in enumerate(sp):
        gaps[num_x,num_y] = gap_A2((x,y))
        
mylib.save_object(gaps,'Jupyter022.002')

In [ ]:
# want to make the data of the same format that I did for the other A2 colour plot
an =  mylib.load_object('Jupyter022.001')
thetas = np.asarray([float(key[6:]) for key in an.keys()])
reorder = thetas.argsort()
thetas = thetas[reorder]
last_probs = np.asarray([an['theta='+str(theta)].problem_x0_prob[-1] for theta in thetas])
obj = {'thetas': thetas, 'last_probs': last_probs}
mylib.save_object(obj,'Jupyter022.003')


In [ ]:
plt.figure()
plt.plot(thetas*(90*2/np.pi), last_probs)
plt.ylabel(r'$P$ (success)')
plt.xlabel(r'$\theta$ in degrees')
plt.show()

In [ ]:
max_success_theta = thetas[last_probs == max(last_probs)][0]
an['theta=%s'%max_success_theta].show_results()